connect the temperature and humidity dataset to the soil moisture dataset

In [20]:
import pandas as pd
import datetime
import obspy as op
import numpy as np
from scipy import interpolate

In [2]:
soil_M = pd.read_csv("Fnames_soilM_interp.csv")

In [3]:
temp = pd.read_csv("Temperature_humidity.dat")

In [4]:
temp.head()

,Daytime,Temphigh,Humidhigh,Humidlow,Battvolt,Pressure,Scaled_temp,Scaled_pressure,hour_of_day
0,2016-07-28 00:00:00,16.06,75.05,0,12.39,1020,0.505827,0.963173,0
1,2016-07-28 01:00:00,15.40,74.29,0,12.38,1020,0.485039,0.963173,1
2,2016-07-28 02:00:00,13.58,77.73,0,12.37,1020,0.427717,0.963173,2
3,2016-07-28 03:00:00,12.40,78.49,0,12.36,1020,0.390551,0.963173,3
4,2016-07-28 04:00:00,12.13,78.54,0,12.35,1020,0.382047,0.963173,4


In [7]:
temp.drop(["Scaled_temp","Scaled_pressure"],axis=1,inplace=True)

In [8]:
#Find the start of the temperature experiment
stime = op.UTCDateTime(temp['Daytime'][0])

In [9]:
temp['tsince_start'] = temp['Daytime'].apply(lambda x: op.UTCDateTime(x)-stime)

In [10]:
temp.head()

,Daytime,Temphigh,Humidhigh,Humidlow,Battvolt,Pressure,hour_of_day,tsince_start
0,2016-07-28 00:00:00,16.06,75.05,0,12.39,1020,0,0.0
1,2016-07-28 01:00:00,15.40,74.29,0,12.38,1020,1,3600.0
2,2016-07-28 02:00:00,13.58,77.73,0,12.37,1020,2,7200.0
3,2016-07-28 03:00:00,12.40,78.49,0,12.36,1020,3,10800.0
4,2016-07-28 04:00:00,12.13,78.54,0,12.35,1020,4,14400.0


In [14]:
soil_M.rename(index=str,columns={"tsince_start":"tsince_start_soilM"},inplace=True)

In [18]:
soil_M['tsince_start_temp'] = soil_M['DateTime'].apply(lambda x: op.UTCDateTime(x)+30-stime)

In [21]:
X = np.array(temp['tsince_start'])
for i in range(1,5):
    colname = temp.columns[i]
    print(colname)
    newcolname = colname+'_interp'
    Y = np.array(temp[colname])
    f = interpolate.interp1d(X,Y,kind='linear')
    soil_M[newcolname] = f(np.array(soil_M['tsince_start_temp']))

Temphigh
Humidhigh
Humidlow
Battvolt


In [22]:
soil_M.head()

,file_name,Line,Timestamp,DateTime,tsince_start_soilM,M1_20_Avg_interp,M2_20_Avg_interp,M3_20_Avg_interp,M4_10_Avg_interp,M4_35_Avg_interp,...,M7_20_Avg_interp,M7_35_Avg_interp,M8_20_Avg_interp,M9_20_Avg_interp,M9_35_Avg_interp,tsince_start_temp,Temphigh_interp,Humidhigh_interp,Humidlow_interp,Battvolt_interp
0,./20160805180738_40_Hz_LineC,LineC,20160805180738,2016-08-05 18:07:38,21488.0,0.581,0.61056,0.202,0.533,0.497813,...,0.558187,0.562,0.311,0.417,0.495187,756488.0,27.437489,45.485533,0.0,12.74
1,./20160805180738_40_Hz_LineCSN.mseed,LineCSN,20160805180738,2016-08-05 18:07:38,21488.0,0.581,0.61056,0.202,0.533,0.497813,...,0.558187,0.562,0.311,0.417,0.495187,756488.0,27.437489,45.485533,0.0,12.74
2,./20160805180738_40_Hz_Line2EW.mseed,Line2EW,20160805180738,2016-08-05 18:07:38,21488.0,0.581,0.61056,0.202,0.533,0.497813,...,0.558187,0.562,0.311,0.417,0.495187,756488.0,27.437489,45.485533,0.0,12.74
3,./20160805180738_40_Hz_Line2,Line2,20160805180738,2016-08-05 18:07:38,21488.0,0.581,0.61056,0.202,0.533,0.497813,...,0.558187,0.562,0.311,0.417,0.495187,756488.0,27.437489,45.485533,0.0,12.74
4,./20160805180838_40_Hz_Line2EW.mseed,Line2EW,20160805180838,2016-08-05 18:08:38,21548.0,0.581,0.61026,0.202,0.533,0.497913,...,0.558087,0.562,0.311,0.417,0.495087,756548.0,27.423656,45.512033,0.0,12.74


In [23]:
soil_M.to_csv("Fnames_soilM_temp.csv",index=False)